<a href="https://colab.research.google.com/github/ShagufthaZK/IR_project/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok
!pip install Flask-WTF
!pip install Flask-Bootstrap4

In [ ]:
!pip install tfkit==0.7.21 nlp2go==0.4.12
!pip install git+https://github.com/roemmele/answerquest.git
!pip install torch==1.10.0+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html

In [ ]:
!git clone https://github.com/roemmele/answerquest.git
!bash /content/answerquest/download_models.sh

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!cp -r /content/doc_qa_model /content/gdrive/MyDrive/IR_code
!cp -r /content/qg_augmented_model /content/gdrive/MyDrive/IR_code

In [ ]:
!wget https://github.com/voidful/BDG/releases/download/v1.0/BDG_ANPM.pt
!wget https://github.com/voidful/BDG/releases/download/v1.0/BDG_PM.pt
!wget https://github.com/voidful/BDG/releases/download/v1.0/BDG.pt

In [ ]:
!cp -r /content/BDG_PM.pt /content/gdrive/MyDrive/IR_code
!cp -r /content/BDG_ANPM.pt /content/gdrive/MyDrive/IR_code
!cp -r /content/BDG.pt /content/gdrive/MyDrive/IR_code

In [ ]:
from pyngrok import ngrok
from flask import Flask, render_template , request 
import os
import threading
import pickle
from flask_bootstrap import Bootstrap
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, TextAreaField
from wtforms.validators import DataRequired

from answerquest import QnAPipeline
import nlp2go


In [ ]:
class InputForm(FlaskForm):
    inputText = TextAreaField('Enter Quiz Material', validators=[DataRequired()],render_kw={'rows':'10'})
    submit = SubmitField('Generate Questions')


In [ ]:
def initialise(qg_tokenizer_path, qg_model_path, qa_model_path, BDG_model_path ):
  qna_pipeline = QnAPipeline(
    qg_tokenizer_path=qg_tokenizer_path,
    qg_model_path=qg_model_path,
    qa_model_path=qa_model_path
    # qg_tokenizer_path="qg_augmented_model/gpt2_tokenizer_vocab/",
    # qg_model_path="qg_augmented_model/qg_augmented_model.pt",
    # qa_model_path="doc_qa_model/checkpoint-59499/"
  )

  bdg_pm_model = nlp2go.Model(BDG_model_path)
  # bdg_pm_model = nlp2go.Model('./BDG_PM.pt')

  return qna_pipeline, bdg_pm_model


def func(input_text):
    qna_pipeline, bdg_pm_model = initialise("/content/gdrive/MyDrive/IR_code/qg_augmented_model/gpt2_tokenizer_vocab/", "/content/gdrive/MyDrive/IR_code/qg_augmented_model/qg_augmented_model.pt", "/content/gdrive/MyDrive/IR_code/doc_qa_model/checkpoint-59499", '/content/gdrive/MyDrive/IR_code/BDG_PM.pt')

    (sent_idxs,questions,answers) = qna_pipeline.generate_qna_items(
        input_text,
        filter_duplicate_answers=True,
        filter_redundant=True,
        sort_by_sent_order=True)

    # sent_idxs, questions, answers
    distractors = []

    result = []

    for a,b,c in zip(sent_idxs, questions, answers):
        print(a)

        strI = input_text + " [SEP] " + b + " [SEP] " + c
        input_json = {
        "input": strI
        }

        x=bdg_pm_model.predict(input_json)
        new_entry = dict()
        new_entry['question']=b
        new_entry['line_no']=a
        new_entry['ans']=c
        new_entry['distractor']=x['result'][0]
        result.append(new_entry)

    return result

In [ ]:

app = Flask(__name__,template_folder='/content/gdrive/MyDrive/IR_code/templates',static_folder='/content/gdrive/MyDrive/IR_code/static')
ngrok.set_auth_token("*******************************")#TODO: add this auth token here
port = 5000
public_url = ngrok.connect(port).public_url
app.config["BASE_URL"] = public_url
app.config['SECRET_KEY'] = '***************************'#TODO: add a secret key here
Bootstrap(app)

In [ ]:
# qna_pipeline= None
# bdg_pm_model=None

# @app.before_first_request
# def activate_job():
#   qna_pipeline, bdg_pm_model = initialise("/content/gdrive/MyDrive/IR_code/qg_augmented_model/gpt2_tokenizer_vocab/", "/content/gdrive/MyDrive/IR_code/qg_augmented_model/qg_augmented_model.pt", "/content/gdrive/MyDrive/IR_code/doc_qa_model/checkpoint-59499", '/content/gdrive/MyDrive/IR_code/BDG_PM.pt')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/quizgen',methods=["GET", "POST"])
def quizgen():
    form = InputForm()
    qa = []
    if form.validate_on_submit():
        inputData = form.inputText.data
        qa = func(inputData)
        print(qa)
        #return render_template('quizgen.html',form=form,result=qa)
    return render_template('quizgen.html',form=form,results=qa)
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
app.run()
